In [1]:
! pip install dspy-ai PyPDF2


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
# update requirements.txt
! pip freeze > requirements.txt

In [1]:
import atefar
import dspy

/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def configure_lm():
    """Configure and return a language model."""
    lm = dspy.Claude("claude-3-5-sonnet-20240620", api_key="***REMOVED***")
    dspy.configure(lm=lm)
    return lm

configure_lm()

In [3]:
from atefar.sig_chains import full_paper_chain

In [4]:
from atefar import pdf_utils
pdf_text = atefar.pdf_utils.extract_text_from_pdf("papers/94cifar.pdf")

In [5]:
solver = full_paper_chain(paper_content=pdf_text)

In [6]:
x = solver.forward()

Running step self = Predict(StringSignature(paper_content -> rationale, title, abstract_plus
    instructions='Given the fields , produce the fields .'
    paper_content = Field(annotation=str required=True json_schema_extra={'desc': "\n    The full text content of an AI research paper. Our eventual goal is to produce tasks the correspond to replicating parts of this paper \n    which can be used to evaluate the capabilities of researchers and developers. In other words, we want to identify one or more methods\n    that the paper explores, as well as how the efficacy of those methods was measured (metrics). Then we want to develop a task wherein we will\n    ask the subject to replicate some method explored in the paper in the form of a python function. For each task, we will eventually produce: \n    a description of the function to implement, the inputs to that function, the expected outputs from that function, (if applicable) a baseline\n    implementation of the function to be give

In [7]:
x.keys()

dict_keys(['title_abstract_plus', 'quantitative_results_json', 'core_ideas_json', 'metrics_json', 'hw_agnostic_metrics_json', 'baseline_methods_json', 'experimental_methods_json', 'method_metric_results', 'task_candidates_json', 'task_prerequisites_json', 'task_eval_instructions_json', 'task_eval_baseline_implementation_json', 'task_eval_correctness_scoring_json', 'task_eval_metric_scoring_json', 'task_eval_combined_scoring_json', 'task_setup_script'])

In [9]:
x['task_setup_script']['task_setup_script']

'Here\'s the task setup script based on the provided information:\n\n```bash\n#!/bin/bash\n\n# Help message\nif [ "$#" -ne 1 ]; then\n    echo "Usage: $0 <output_directory>"\n    exit 1\nfi\n\n# Set variables\nOUTPUT_DIR="$1"\n\n# Create output directory if it doesn\'t exist\nmkdir -p "$OUTPUT_DIR"\n\n# Function to create files\ncreate_files() {\n    # Create instructions.txt\n    cat > "$OUTPUT_DIR/instructions.txt" << EOL\nImplement the alternating flip augmentation method as described in the paper. Your implementation should:\n1. Take a batch of images and the current epoch number as inputs.\n2. For the first epoch, randomly flip 50% of the images horizontally.\n3. For subsequent epochs, deterministically flip images based on whether they were flipped in the first epoch:\n   - On even epochs, flip only those images that were not flipped in the first epoch.\n   - On odd epochs, flip only those images that were flipped in the first epoch.\n4. Use a pseudorandom function based on image

In [1]:
import dspy
import atefar
from dataclasses import dataclass
from typing import Any
# from atefar.sig_types import SigChain, DspyStep


def solve_sigchain(chain: SigChain, inputs=dict[str, str]) -> tuple[dict[str, Any], dict[str, Any]]:
    input_sigs = chain.inputs.copy()
    input_vals = inputs.copy()
    solvers = dict()
    results = dict()
    for step in chain.steps:
        try:
            assert isinstance(step, DspyStep), f"Expected SigStep, got {type(step)}: {step}"
            step_name = "_".join([o.name for o in step.outputs])
            step_inputs_str = ", ".join([i.name for i in input_sigs])
            step_outputs_str = ", ".join([o.name for o in step.outputs])
            print(f"Running step {step_name}")
            print(f"  Inputs: {step_inputs_str}")
            print(f"  Outputs: {step_outputs_str}")
            class NextSig(dspy.Signature):
                pass
            for i in input_sigs:
                NextSig = NextSig.append(i.name, dspy.InputField(desc=i.desc))
            for o in step.outputs:
                NextSig = NextSig.append(o.name, dspy.OutputField(desc=o.desc))
            step_solver = step.module(NextSig, **(step.module_kwargs or {}))
            result = step_solver(**input_vals)
            for o in step.outputs:
                input_vals[o.name] = result[o.name]
            input_sigs.extend(step.outputs)
            solvers[step_name] = step_solver
            results[step_name] = result
        except Exception as e:
            print(f"Error running step {step_name}: {e}; returning early")
    return solvers, results

def construct_solvers(chain: SigChain) -> dict[str, Any]:
    solvers = dict()
    input_sigs = chain.inputs.copy()

    for step in chain.steps:
        assert isinstance(step, DspyStep), f"Expected SigStep, got {type(step)}: {step}"
        step_name = "_".join([o.name for o in step.outputs])

        class NextSig(dspy.Signature):
            pass

        for i in input_sigs:
            NextSig = NextSig.append(i.name, dspy.InputField(desc=i.desc))
        for o in step.outputs:
            NextSig = NextSig.append(o.name, dspy.OutputField(desc=o.desc))

        step_solver = step.module(NextSig, **(step.module_kwargs or {}))
        solvers[step_name] = step_solver
        input_sigs.extend(step.outputs)

    return solvers

def execute_solvers(chain: SigChain, solvers: dict[str, Any], inputs: dict[str, str]) -> dict[str, Any]:
    input_vals = inputs.copy()
    results = dict()

    for step in chain.steps:
        step_name = "_".join([o.name for o in step.outputs])
        step_inputs_str = ", ".join([i.name for i in chain.inputs])
        step_outputs_str = ", ".join([o.name for o in step.outputs])

        print(f"Running step {step_name}")
        print(f"  Inputs: {step_inputs_str}")
        print(f"  Outputs: {step_outputs_str}")

        step_solver = solvers[step_name]
        result = step_solver(**input_vals)

        for o in step.outputs:
            input_vals[o.name] = result[o.name]

        results[step_name] = result

    return results

/Users/jaidhyani/Desktop/atefar/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from atefar.sig_chains import full_paper_chain
solvers, results = solve_sigchain(full_paper_chain, {"paper_content": pdf_text})

Running step title_abstract_plus
  Inputs: paper_content
  Outputs: title, abstract_plus
Running step quantitative_results_json
  Inputs: paper_content, title, abstract_plus
  Outputs: quantitative_results_json
Running step core_ideas_json
  Inputs: paper_content, title, abstract_plus, quantitative_results_json
  Outputs: core_ideas_json
Running step metrics_json
  Inputs: paper_content, title, abstract_plus, quantitative_results_json, core_ideas_json
  Outputs: metrics_json
Running step hw_agnostic_metrics_json
  Inputs: paper_content, title, abstract_plus, quantitative_results_json, core_ideas_json, metrics_json
  Outputs: hw_agnostic_metrics_json
Running step baseline_methods_json
  Inputs: paper_content, title, abstract_plus, quantitative_results_json, core_ideas_json, metrics_json, hw_agnostic_metrics_json
  Outputs: baseline_methods_json
Running step experimental_methods_json
  Inputs: paper_content, title, abstract_plus, quantitative_results_json, core_ideas_json, metrics_json, 

In [13]:
x = solvers['task_setup_script']

In [24]:
full_paper_chain.steps[-1].outputs[0].desc

'\n    A bash script that sets up the environment for running the task evaluation functions in a Linux environment. The script should:\n    1. Take one positional argument: the path to the directory where the task evaluation code will be located\n    2. Create the following files in the specified directory:\n       - instructions.txt: A text file containing the detailed instructions for each task\n       - solution.py: A python file containing the baseline implementations\n       - scoring.py: A python file containing the scoring functions\n       - requirements.txt: A text file listing all Python package dependencies\n    3. Set up a Python virtual environment and install required packages\n    4. Include error handling and logging\n    5. Add a help message when run without arguments\n\n    The script should be self-contained.\n    It should not contain any placeholders. Do not assume that any environment variables are set. \n    Include comments for clarity. \n    Do not include any

In [9]:
results['task_setup_script']['task_setup_script']

'Here is the bash script to set up the task evaluation environment:\n\n```bash\n#!/bin/bash\n\n# Help message\nif [ "$#" -ne 1 ]; then\n    echo "Usage: $0 <output_directory>"\n    exit 1\nfi\n\n# Set variables\nOUTPUT_DIR="$1"\n\n# Create output directory if it doesn\'t exist\nmkdir -p "$OUTPUT_DIR"\n\n# Function to create files\ncreate_files() {\n    # Create instructions.txt\n    cat > "$OUTPUT_DIR/instructions.txt" << EOL\nImplement the alternating flip augmentation method as described in Section 3.6 of the paper. Your implementation should:\n1. Take a batch of images and the current epoch number as input.\n2. For the first epoch, randomly flip 50% of the images horizontally.\n3. For subsequent epochs, deterministically flip images based on whether they were flipped in the first epoch:\n   - On even epochs (2, 4, 6, ...), flip only those images that were not flipped in the first epoch.\n   - On odd epochs (3, 5, 7, ...), flip only those images that were flipped in the first epoch.\

In [5]:
records = {}
records["v7"] = (solvers, results)

In [5]:
for k in results.keys():
    for kk in results[k].keys():
        if kk == "rationale":
            continue
        print(f"{kk}:\n{results[k][kk]}\n\n")
    print ("\n\n====================\n\n")

title:
94% on CIFAR-10 in 3.29 Seconds on a Single GPU


abstract_plus:
Title: 94% on CIFAR-10 in 3.29 Seconds on a Single GPU

Abstract Plus: This paper introduces fast training methods for the CIFAR-10 dataset, achieving 94% accuracy in 3.29 seconds, 95% in 10.4 seconds, and 96% in 46.3 seconds on a single NVIDIA A100 GPU. The authors propose a derandomized variant of horizontal flipping augmentation called "alternating flip," which improves performance over standard random flipping in all cases where flipping is beneficial. The methods build upon prior work, incorporating techniques such as a modified network architecture, initialization, optimizer, and frozen patch-whitening layer. The paper presents a detailed analysis of the components contributing to the speedup and demonstrates the effectiveness of the alternating flip method across various scenarios.

Additional high-level summaries:
1. The paper provides a comprehensive breakdown of the techniques used to achieve fast trainin

In [6]:
x = results['task_setup_script']

In [8]:
len(x.completions)

1

In [9]:
help(x)

Help on Prediction in module dspy.primitives.prediction object:

class Prediction(dspy.primitives.example.Example)
 |  Prediction(*args, **kwargs)
 |  
 |  Method resolution order:
 |      Prediction
 |      dspy.primitives.example.Example
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_completions(list_or_dict, signature=None) from builtins.type
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  completions
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from dspy.primitives.example.Example:
 |  
 |  __contains__(self,

In [18]:
solvers['task_setup_script']

self = Predict(StringSignature(paper_content, title, abstract_plus, quantitative_results_json, core_ideas_json, metrics_json, hw_agnostic_metrics_json, baseline_methods_json, experimental_methods_json, method_metric_results, task_candidates_json, task_prerequisites_json, task_eval_instructions_json, task_eval_baseline_implementation_json, task_eval_correctness_scoring_json, task_eval_metric_scoring_json, task_eval_combined_scoring_json -> rationale, task_setup_script
    instructions='Given the fields , produce the fields .'
    paper_content = Field(annotation=str required=True json_schema_extra={'desc': "\n    The full text content of an AI research paper. Our eventual goal is to produce tasks the correspond to replicating parts of this paper \n    which can be used to evaluate the capabilities of researchers and developers. In other words, we want to identify one or more methods\n    that the paper explores, as well as how the efficacy of those methods was measured (metrics). Then w

In [ ]:
dspy.ChainOfThought

In [13]:
import json
with open('aa', 'w') as f:
    json.dump(x.toDict(), f, indent=2)

In [ ]:
class PaperAnalysis(dspy.Signature):
    """Analyze a research paper and provide core information."""

    paper_content = dspy.InputField(desc="The full text content of the research paper")

    title = dspy.OutputField(desc="The title of the paper")
    summary = dspy.OutputField(desc="A concise summary of the paper's main contributions")
    core_ideas = dspy.OutputField(desc="The core idea(s) of the paper")
    methods = dspy.OutputField(desc="Key methods or strategies proposed in the paper")
    metrics = dspy.OutputField(desc="Primary metrics or evaluation criteria used in the paper")
    requirements = dspy.OutputField(desc="Key requirements for implementing and evaluating the paper's methods")

In [14]:

def analyze_paper(pdf_path, model_name="gpt-3.5-turbo"):
    """Analyze a research paper given its PDF path."""

    # Configure LM
    lm = configure_lm()

    # Extract text from PDF
    paper_content = pdf_utils.extract_text_from_pdf(pdf_path)
 
    # Initialize and compile the analyzer
    analyzer = dspy.ChainOfThought(paper_analysis.PaperAnalysis)
    # Analyze the paper
    result = analyzer(paper_content=paper_content)

    return result

In [15]:
result=analyze_paper("papers/94cifar.pdf")

In [22]:
result.keys()

['rationale', 'summary', 'core_ideas', 'methods', 'metrics', 'requirements']

In [26]:
result.summary

str

In [24]:
result.methods

'1. Modified network architecture based on prior work, with some adjustments to layer sizes and initialization.\n2. Frozen patch-whitening initialization for the first convolutional layer.\n3. Identity initialization for subsequent convolutional layers.\n4. Optimization tricks including increased learning rate for BatchNorm biases and Lookahead optimization.\n5. Multi-crop evaluation using six augmented views of each test image.\n6. Alternating flip augmentation technique.\n7. Compilation using torch.compile for improved GPU utilization.'

# New Version

In [24]:
results['title_abstract_plus']['title']

'94% on CIFAR-10 in 3.29 Seconds on a Single GPU'

In [28]:
for r in results:
    print(r)
    print('--------')
    for q in results[r]:
        print(f"  {q}: {results[r][q]}")
        print('--------')
    print('\n\n====================\n\n')

title_abstract_plus
--------
  rationale: produce the abstract_plus. We should:

1. Identify the key points from the paper content, focusing on:
   - The main objective of the research
   - The methods used
   - The results achieved
   - Any novel contributions

2. Summarize these key points concisely to form the abstract.

3. Add any additional high-level summaries that capture interesting aspects not covered in the abstract.

Here's the abstract_plus based on the paper content:

Abstract:
This paper introduces fast training methods for the CIFAR-10 dataset, achieving 94% accuracy in 3.29 seconds, 95% in 10.4 seconds, and 96% in 46.3 seconds on a single NVIDIA A100 GPU. The authors propose a derandomized variant of horizontal flipping augmentation called "alternating flip," which improves performance over standard random flipping in all cases where flipping is beneficial. The methods build upon prior work, incorporating techniques such as a modified network architecture, initializatio

In [29]:
help(results['title_abstract_plus'])

Help on Prediction in module dspy.primitives.prediction object:

class Prediction(dspy.primitives.example.Example)
 |  Prediction(*args, **kwargs)
 |  
 |  Method resolution order:
 |      Prediction
 |      dspy.primitives.example.Example
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_completions(list_or_dict, signature=None) from builtins.type
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  completions
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from dspy.primitives.example.Example:
 |  
 |  __contains__(self,

In [36]:
results['title_abstract_plus'].keys()

['rationale', 'title', 'abstract_plus']

In [38]:
results['title_abstract_plus']['title']

'94% on CIFAR-10 in 3.29 Seconds on a Single GPU'